## Ejercicio 1: Preliminares y datos perdidos

### Genere los objetos SparkConf y SparkContext .

In [1]:
#inspeccionamos el SparkContex
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1639255716588_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

### Genere los objetos SparkConf y SparkContext .

In [2]:
#importamos la data
hpc = sc.textFile('s3://bigdata-desafio/challenges/u4lec1/household_power_consumption.txt')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#la inspeccionamos
hpc.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3', '16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000']

### El primer registro del archivo corresponde a las etiquetas asociadas a cada columna. Elimínelo de su RDD.

In [4]:
#obtenemos el listado de columnas
hpc_columnas = hpc.first()
hpc_columnas

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Date;Time;Global_active_power;Global_reactive_power;Voltage;Global_intensity;Sub_metering_1;Sub_metering_2;Sub_metering_3'

In [5]:
#creamos un RDD sin los nombres de las columnas
hpc_data = hpc.filter(lambda x: x != hpc_columnas)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#lo revisamos
hpc_data.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['16/12/2006;17:24:00;4.216;0.418;234.840;18.400;0.000;1.000;17.000']

### Cada campo dentro del registro está separado por un ; . Separe cada campo del registro por éste caracter.

In [7]:
#creamos un RDD con los datos separados por ;
hcp_data_splitted = hpc_data.map(lambda x: x.split(';'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#lo revisamos
hcp_data_splitted.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['16/12/2006', '17:24:00', '4.216', '0.418', '234.840', '18.400', '0.000', '1.000', '17.000']]

### Reporte las siguientes métricas:

### * La cantidad total de registros antes de eliminar los datos perdidos.

In [9]:
#contamos la data total incluyendo los datos perdidos
total_datos = hcp_data_splitted.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#lo reportamos
print(f"Cantidad total de registros antes de eliminar los datos perdidos: ==> \033[1m{total_datos}\033[0m")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cantidad total de registros antes de eliminar los datos perdidos: ==> 2075259

### * La cantidad total de registros nulos.

In [11]:
#almacenamos los registros con datos nulos y los contamos
datos_perdidos = hcp_data_splitted.filter(lambda x: x.count('?') > 0 )
total_datos_nulos = datos_perdidos.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#reportamos el total
print(f"Cantidad total de datos nulos: ==> \033[1m{total_datos_nulos}\033[0m")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cantidad total de datos nulos: ==> 25979

###  * La cantidad de registros nulos por año. ¿En qué año hubo más registros nulos?

In [13]:
#creamos un RDD con los datos nulos por año
datos_nulos_ano = datos_perdidos\
    .map(lambda x: (int(x[0].split('/')[-1]), 1))\
    .reduceByKey(lambda x, y : x + y)\
    .sortBy(lambda x: x[1], ascending=False)\
    .collect()
#obtenemos el año con mas datos nulos y su total de datos nulos
anno_con_mas = datos_nulos_ano[0][0]
anno_con_mas_nulos = datos_nulos_ano[0][1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#reportamos el año con mas datos nulos y el total de ellos
print(f"El año con más registros nulos fue \033[1m{anno_con_mas}\033[0m, con \033[1m{anno_con_mas_nulos}\033[0m registros nulos.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

El a?o con m?s registros nulos fue 2010, con 17629 registros nulos.

### * La cantidad de registros nulos por mes para el año con una mayor cantidad de registros nulos. ¿En qué mes hubo más registros nulos?

In [15]:
#creamos un RDD con los meses y total de registros nulos, filtrado por el año con mas datos nulos
datos_nulos_ano_mes = datos_perdidos\
    .map(lambda x: (int(x[0].split('/')[-1]), int(x[0].split('/')[1]), 1))\
    .filter(lambda x: x[0] == anno_con_mas)\
    .map(lambda x: (x[1], x[2]))\
    .reduceByKey(lambda x, y: x + y)\
    .sortBy(lambda x: x[1], ascending=False)\
    .collect()
#obtenemos el mes con más datos nulos y el total de estos
mes_con_mas = datos_nulos_ano_mes[0][0]
mes_con_mas_nulos = datos_nulos_ano_mes[0][1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#reportamos el mes del año con mas datos nulos, y el total de estos
print(f"El mes con más registros nulos, para el año \033[1m{anno_con_mas}\033[0m, fue el mes \033[1m{mes_con_mas}\033[0m, con \033[1m{mes_con_mas_nulos}\033[0m registros nulos.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

El mes con m?s registros nulos, para el a?o 2010, fue el mes 8, con 7226 registros nulos.

## Ejercicio 2: Preparación de los datos

### * Genere un objeto donde se encuentren sólo aquellos datos sin registros perdidos.

In [17]:
#creamos un RDD sin datos nulos
datos_completos = hcp_data_splitted.filter(lambda x: x.count('?') == 0 )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#lo inspeccionamos
datos_completos.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['16/12/2006', '17:24:00', '4.216', '0.418', '234.840', '18.400', '0.000', '1.000', '17.000']]

### * Con el objeto sin datos perdidos, genere un objeto RDD donde los registro tengan la siguiente estructura:
`
[(2006, 12, 18, 4.464, 0.136, 234.66, 19.0, 0.0, 37.0),
(2006, 12, 18, 1.944, 0.084, 236.56, 8.2, 0.0, 2.0),
(2006, 12, 9, 3.706, 0.062, 237.81, 15.6, 1.0, 38.0),
(2006, 12, 1, 0.204, 0.0, 244.48, 0.8, 0.0, 0.0), 
(2006, 12, 10, 1.34, 0.092, 238.72, 5.6, 0.0, 0.0)]
`
### donde cada registro tendrá la siguiente composición:
`(año, mes, dia, global_activity_power, global_reactive_power, voltage, global_intensity, submetering_1, submetering_2, submetering_3)`

### * Por defecto todos los datos dentro de este objeto serán considerados como strings. Conviértalos al tipo de dato pertinente ( int o float ).

In [19]:
#creamos un RDD con los datos formateados acorde a lo solicitado
datos_formateados = datos_completos\
    .map(lambda x: (x[0].split('/'), float(x[2]), float(x[3]), float(x[4]), float(x[5]), float(x[6]), float(x[7]), float(x[8])))\
    .map(lambda x: (int(x[0][2]), int(x[0][1]), int(x[0][0]), x[1], x[2], x[3], x[4], x[5], x[6], x[7]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### * Devuelva con collect las primeras 5 observaciones.

In [20]:
#reportamos lo solicitado
datos_formateados.collect()[:5]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(2006, 12, 16, 4.216, 0.418, 234.84, 18.4, 0.0, 1.0, 17.0), (2006, 12, 16, 5.36, 0.436, 233.63, 23.0, 0.0, 1.0, 16.0), (2006, 12, 16, 5.374, 0.498, 233.29, 23.0, 0.0, 2.0, 17.0), (2006, 12, 16, 5.388, 0.502, 233.74, 23.0, 0.0, 1.0, 17.0), (2006, 12, 16, 3.666, 0.528, 235.68, 15.8, 0.0, 1.0, 17.0)]

## Ejercicio 3: Patrones globales

### * Extraiga la media, desviación estándar e intervalos de confianza (a $\pm.5$ desviaciones estándar) para las columnas `global_activity_power`, `global_reactive_power`, `voltage` y `global_intensity`

### * Reporte sus resultados.

In [21]:
#creamos un diccionario para almacenar los datos solicitados
columnas = [
    'global_activity_power',
    'global_reactive_power',
    'voltage',
    'global_intensity'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#obtenemos el total de datos no perdidos
total_formateados = datos_formateados.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#importamos la libreria para raices cuadradas
from math import sqrt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#calculamos los patrones
patrones_globales = {}

texto = '\033[1mPATRONES GLOBALES\033[0m\n'

#iteramos sobre las columnas
for i, columna in enumerate(columnas):
    texto += f"\n==> Columna '\033[1m{columna}\033[0m'\n"
    patrones_globales[columna] = {}
    #calculamos la media
    media_tmp = datos_formateados\
        .map(lambda x: x[i+3])\
        .reduce(lambda x, y: x+y)\
        / total_formateados
    texto += f"    ==> \x1b[4mMedia\033[0m :\t\t\t\t{media_tmp}\n"
    #almacenamos la media para el patron
    patrones_globales[columna]['media'] = media_tmp
    
    #calculamos la desviacion estandar
    desviacion_tmp = sqrt(datos_formateados\
        .map(lambda x: x[i+3])\
        .map(lambda x: (x-media_tmp)**2)\
        .reduce(lambda x,y: x+y)\
        / total_formateados)
    texto += f"    ==> \x1b[4mDesviacion estandar\033[0m :\t\t{desviacion_tmp}\n"
    #almacenamos la desviacion para el patron
    patrones_globales[columna]['desviacion'] = media_tmp
    
    #calculamos el intervalo de confianza
    intervalo_minimo_tmp = media_tmp-(desviacion_tmp/2) 
    intervalo_maximo_tmp = media_tmp+(desviacion_tmp/2)
    texto += f"    ==> \x1b[4mIntervalo de confianza\033[0m :\t[{intervalo_minimo_tmp}"
    texto += f" .. {intervalo_maximo_tmp}]\n"
    #almacenamos el intervalo para el patron
    patrones_globales[columna]['intervalo_confianza_minimo'] = intervalo_minimo_tmp
    patrones_globales[columna]['intervalo_confianza_maximo'] = intervalo_maximo_tmp

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
#reportamos lo solicitado
print(texto)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATRONES GLOBALES

==> Columna 'global_activity_power'
    ==> Media :				1.0916150365007313
    ==> Desviacion estandar :		1.0572939031266517
    ==> Intervalo de confianza :	[0.5629680849374055 .. 1.6202619880640572]

==> Columna 'global_reactive_power'
    ==> Media :				0.12371447630388727
    ==> Desviacion estandar :		0.11272195204774406
    ==> Intervalo de confianza :	[0.06735350028001524 .. 0.1800754523277593]

==> Columna 'voltage'
    ==> Media :				240.83985797455423
    ==> Desviacion estandar :		3.2399858884913204
    ==> Intervalo de confianza :	[239.21986503030857 .. 242.4598509187999]

==> Columna 'global_intensity'
    ==> Media :				4.627759310588489
    ==> Desviacion estandar :		4.444395175415986
    ==> Intervalo de confianza :	[2.4055617228804964 .. 6.849956898296482]

## Ejercicio 4: Patrón temporal

### * Reporte el promedio de `global_activity_power` a nivel mensual y anual. Genere un objeto con la acción `collectAsmap()` del RDD procesado.

In [26]:
#creamos el RDD
global_activity_power_means = datos_formateados\
    .keyBy(lambda x: (x[0], x[1]))\
    .mapValues(lambda x: (x[3], 1))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .mapValues(lambda x: (x[0]/x[1]))\
    .sortByKey(ascending=False)\

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
#lo reportamos
global_activity_power_means.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 11), 1.196854466897542), ((2010, 10), 1.1633986872466457), ((2010, 9), 0.9564424307878864), ((2010, 8), 0.5907781579088224), ((2010, 7), 0.7210679450704021), ((2010, 6), 0.9696148895782476), ((2010, 5), 1.0952843925715077), ((2010, 4), 1.0272951688695853), ((2010, 3), 1.1300752821908384), ((2010, 2), 1.3758546058832082)]

In [28]:
#generamos el RDD como collectAsMap
gap_asm = global_activity_power_means.collectAsMap()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#lo revisamos
gap_asm[(2010, 11)]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.196854466897542

### * Reporte todas aquellos registros que presenten un promedio de global_activity_power superior a .5 desviaciones estándares. ¿A qué meses y años corresponden?

In [30]:
#creamos una lista con los resultados solicitados
sobre_intervalo_confianza = [(k, v, gap_asm[k,v]) for k, v in gap_asm if gap_asm[k,v] > patrones_globales['global_activity_power']['intervalo_confianza_maximo']]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
#reportamos lo solicitado
texto2 = f"Registros con promedio de 'global_activity_power' \033[1msobre\033[0m el intervalo de confianza de "
texto2 += f"\033[1m{patrones_globales['global_activity_power']['intervalo_confianza_maximo']}\033[0m:\n"
print(texto2)
for i in sobre_intervalo_confianza:
    print(f"\x1b[4maño\033[0m: \033[1m{i[0]}\033[0m, \x1b[4mmes\033[0m: \033[1m{i[1]}\033[0m, \x1b[4mpromedio\033[0m: \033[1m{i[2]}\033[0m\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros con promedio de 'global_activity_power' sobre el intervalo de confianza de 1.6202619880640572:

a?o: 2007, mes: 12, promedio: 1.6264739353480613

a?o: 2006, mes: 12, promedio: 1.9012951073117563

### * Reporte todos aquellos registros que presenten un promedio de global_activity_power inferior a .5 desviaciones estándares. ¿A qué meses y años corresponden?

In [32]:
#creamos una lista con los resultados solicitados
bajo_intervalo_confianza = [(k, v, gap_asm[k,v]) for k, v in gap_asm if gap_asm[k,v] < patrones_globales['global_activity_power']['intervalo_confianza_minimo']]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#reportamos lo solicitado
texto2 = f"Registros con promedio de 'global_activity_power' \033[1mbajo\033[0m el intervalo de confianza de "
texto2 += f"\033[1m{patrones_globales['global_activity_power']['intervalo_confianza_minimo']}\033[0m:\n"
print(texto2)
for i in bajo_intervalo_confianza:
    print(f"\x1b[4maño\033[0m: \033[1m{i[0]}\033[0m, \x1b[4mmes\033[0m: \033[1m{i[1]}\033[0m, \x1b[4mpromedio\033[0m: \033[1m{i[2]}\033[0m\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros con promedio de 'global_activity_power' bajo el intervalo de confianza de 0.5629680849374055:

a?o: 2008, mes: 8, promedio: 0.27648819391553775

## Ejercicio 5: Consumo energético general

### * El siguiente objetivo es calcular el consumo energético general para cada registro en el RDD. La fórmula se detalla a continuación:

$$ConsumoEnergeticoGeneral = Global Activity Power \dfrac{1000}{60} - SubMetering1 - SubMetering2 - SubMetering3$$

### * Implemente la función para todos los registros.

In [34]:
#creamos nuestra funcion
def ceg(global_activity_power, sub_metering_1, sub_metering_2, sub_metering_3):
    return [global_activity_power*(1000/60) - sub_metering_1 - sub_metering_2 - sub_metering_3]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
#la implementamos para todos los registros
consumo_energetico_general = datos_formateados\
    .map(lambda x: tuple(list(x) + ceg(x[3], x[7], x[8], x[9])) )\


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
#revisamos
consumo_energetico_general.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(2006, 12, 16, 4.216, 0.418, 234.84, 18.4, 0.0, 1.0, 17.0, 52.26666666666668)]

### * Genere el RDD con el consumo energético general promedio por Año, Mes y Día.

In [37]:
#creamos el RDD
ceg_amd_promedio = consumo_energetico_general\
    .keyBy(lambda x: (x[0], x[1], x[2]))\
    .mapValues(lambda x: (x[-1], 1))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .mapValues(lambda x: (x[0]/x[1]))\
    .sortByKey(ascending=False)\

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
#revisamos
ceg_amd_promedio.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 11, 26), 10.568276590129326), ((2010, 11, 25), 11.999120370370347)]

### * Reporte el consumo energético general para todos los días de Octubre del 2008.

In [39]:
#filtramos por 2008 y octubre
ceg_dias_octubre_2018 = ceg_amd_promedio\
    .filter(lambda x: x[0][0] == 2008 and x[0][1] == 10)\
    .sortBy(lambda x: x[0][2])\
    .collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
#reportamos
print(f"\033[1mConsumo Energetico General\033[0m para el mes de \033[1mOctubre\033[0m de \033[1m2008\033[0m:\n")
for i in ceg_dias_octubre_2018:
    dia = i[0][2]
    if dia < 10:
        dia_txt = f"0{dia}"
    else:
        dia_txt = f"{dia}"
    print(f"\x1b[4mDia\033[0m: \033[1m{dia_txt}\033[0m - \x1b[4mCEG\033[0m: \033[1m{i[1]}\033[0m")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Consumo Energetico General para el mes de Octubre de 2008:

Dia: 01 - CEG: 8.57574074074073
Dia: 02 - CEG: 9.024560185185186
Dia: 03 - CEG: 8.53773148148149
Dia: 04 - CEG: 12.739074074074084
Dia: 05 - CEG: 10.773958333333315
Dia: 06 - CEG: 8.952569444444435
Dia: 07 - CEG: 12.406342592592578
Dia: 08 - CEG: 9.159444444444452
Dia: 09 - CEG: 12.0231712962963
Dia: 10 - CEG: 8.937939814814817
Dia: 11 - CEG: 8.313263888888887
Dia: 12 - CEG: 11.281249999999998
Dia: 13 - CEG: 9.243935185185197
Dia: 14 - CEG: 10.596944444444446
Dia: 15 - CEG: 11.752962962962973
Dia: 16 - CEG: 11.51157407407408
Dia: 17 - CEG: 11.286643518518511
Dia: 18 - CEG: 15.777662037037043
Dia: 19 - CEG: 13.581643518518502
Dia: 20 - CEG: 9.601087962962959
Dia: 21 - CEG: 13.572407407407413
Dia: 22 - CEG: 10.932291666666671
Dia: 23 - CEG: 12.271689814814822
Dia: 24 - CEG: 10.485949074074057
Dia: 25 - CEG: 7.746552135528486
Dia: 26 - CEG: 4.411018518518521
Dia: 27 - CEG: 4.418425925925917
Dia: 28 - CEG: 10.0045138888889
Dia: 29

## (Opcional) Ejercicio 6: Submetering

### * Identifique el consumo promedio por mes y año para cada medición de submetering (1, 2 y 3).

In [41]:
#creamos el RDD para submetering 1
s1_promedio = datos_formateados\
    .keyBy(lambda x: (x[0], x[1]))\
    .mapValues(lambda x: (x[7], 1))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .mapValues(lambda x: (x[0]/x[1]))\
    .sortByKey(ascending=False)\

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
#revisamos
s1_promedio.take(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 11), 1.2388159836835466), ((2010, 10), 1.0797508904769373), ((2010, 9), 0.9710771013881938), ((2010, 8), 0.33407280697065267), ((2010, 7), 0.39577499495956453)]

In [43]:
#creamos el RDD para submetering 2
s2_promedio = datos_formateados\
    .keyBy(lambda x: (x[0], x[1]))\
    .mapValues(lambda x: (x[8], 1))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .mapValues(lambda x: (x[0]/x[1]))\
    .sortByKey(ascending=False)\

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#revisamos
s2_promedio.take(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 11), 1.1346912486917318), ((2010, 10), 1.3490445574497636), ((2010, 9), 0.8563074572610173), ((2010, 8), 0.7212006200887369), ((2010, 7), 0.7697081027800802)]

In [45]:
#creamos el RDD para submetering 3
s3_promedio = datos_formateados\
    .keyBy(lambda x: (x[0], x[1]))\
    .mapValues(lambda x: (x[9], 1))\
    .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
    .mapValues(lambda x: (x[0]/x[1]))\
    .sortByKey(ascending=False)\

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#revisamos
s3_promedio.take(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 11), 6.631591659286692), ((2010, 10), 7.071596585945025), ((2010, 9), 6.792429470800516), ((2010, 8), 4.28152563211632), ((2010, 7), 4.321602186428907)]

### * Identifique aquellos registros donde el consumo promedio del submetering 3 sea mayor al submetering 1.

In [47]:
#creamos una funcion para obtener lo solicitado
def registros_x_mayor_y(rdd_joined, x, y):
    x_mayor_y = rdd_joined\
        .filter(lambda x: x[1][0] > x[1][1])\
        .sortByKey(ascending=True)\
        .collect()
    #reportamos
    texto = f"Registros donde el \033[1mconsumo promedio del submetering {x} es mayor"
    texto += f" al del submetering {y}\033[0m (total: {len(x_mayor_y)}):\n"
    print(texto)
    for i in x_mayor_y:
        mes = i[0][1]
        if mes < 10:
            mes_txt = f"0{mes}"
        else:
            mes_txt = f"{mes}"
        texto = f"año: \033[1m{i[0][0]}\033[0m - mes \033[1m{mes_txt}\033[0m, promedio s{x}: "
        texto += f"\033[1m{i[1][1]}\033[0m\t-\tpromedio s{y}: \033[1m{i[1][0]}\033[0m"
        print(texto)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
#realizamos el join y revisamos
join_s3_s1 = s3_promedio.join(s1_promedio)
join_s3_s1.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 8), (4.28152563211632, 0.33407280697065267))]

In [49]:
#ejecutamos nuestra funcion
registros_x_mayor_y(join_s3_s1, 3, 1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros donde el consumo promedio del submetering 3 es mayor al del submetering 1 (total: 48):

a?o: 2006 - mes 12, promedio s3: 1.2486358675882139	-	promedio s1: 7.409558021098581
a?o: 2007 - mes 01, promedio s3: 1.2642367489582866	-	promedio s1: 7.383350508535329
a?o: 2007 - mes 02, promedio s3: 1.1802172726821767	-	promedio s1: 6.703556724043851
a?o: 2007 - mes 03, promedio s3: 1.3613432200542126	-	promedio s1: 6.5046484016219
a?o: 2007 - mes 04, promedio s3: 1.065886465536895	-	promedio s1: 4.800339438153862
a?o: 2007 - mes 05, promedio s3: 1.6966173835125449	-	promedio s1: 5.1399641577060935
a?o: 2007 - mes 06, promedio s3: 1.3826725676020113	-	promedio s1: 4.375906573672869
a?o: 2007 - mes 07, promedio s3: 0.9672650475184794	-	promedio s1: 3.478285290615381
a?o: 2007 - mes 08, promedio s3: 0.812474785960823	-	promedio s1: 5.05271415123941
a?o: 2007 - mes 09, promedio s3: 1.2232279272188527	-	promedio s1: 5.24040464836335
a?o: 2007 - mes 10, promedio s3: 0.9681885389130337	-	pro

### * Identifique aquellos registros donde el consumo promedio del submetering 2 sea mayor al submetering 1.

In [50]:
#realizamos el join y revisamos
join_s2_s1 = s2_promedio.join(s1_promedio)
join_s2_s1.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 8), (0.7212006200887369, 0.33407280697065267))]

In [51]:
#ejecutamos nuestra funcion
registros_x_mayor_y(join_s2_s1, 2, 1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros donde el consumo promedio del submetering 2 es mayor al del submetering 1 (total: 29):

a?o: 2006 - mes 12, promedio s2: 1.2486358675882139	-	promedio s1: 2.21498726809749
a?o: 2007 - mes 01, promedio s2: 1.2642367489582866	-	promedio s1: 1.7759308212733544
a?o: 2007 - mes 02, promedio s2: 1.1802172726821767	-	promedio s1: 1.6023612282355275
a?o: 2007 - mes 03, promedio s2: 1.3613432200542126	-	promedio s1: 2.346871569703622
a?o: 2007 - mes 06, promedio s2: 1.3826725676020113	-	promedio s1: 1.6205714020900432
a?o: 2007 - mes 07, promedio s2: 0.9672650475184794	-	promedio s1: 1.252173717675077
a?o: 2007 - mes 08, promedio s2: 0.812474785960823	-	promedio s1: 1.1141467569142498
a?o: 2007 - mes 09, promedio s2: 1.2232279272188527	-	promedio s1: 1.7426038242511228
a?o: 2007 - mes 10, promedio s2: 0.9681885389130337	-	promedio s1: 1.9694878802813747
a?o: 2007 - mes 11, promedio s2: 1.1765128015185888	-	promedio s1: 1.705310431038474
a?o: 2007 - mes 12, promedio s2: 1.6597594032124

### * Identifique aquellos registros donde el consumo promedio del submetering 3 sea mayor al submetering 2.

In [52]:
#realizamos el join y revisamos
join_s3_s2 = s3_promedio.join(s2_promedio)
join_s3_s2.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 8), (4.28152563211632, 0.7212006200887369))]

In [53]:
#ejecutamos nuestra funcion
registros_x_mayor_y(join_s3_s2, 3, 2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros donde el consumo promedio del submetering 3 es mayor al del submetering 2 (total: 48):

a?o: 2006 - mes 12, promedio s3: 2.21498726809749	-	promedio s2: 7.409558021098581
a?o: 2007 - mes 01, promedio s3: 1.7759308212733544	-	promedio s2: 7.383350508535329
a?o: 2007 - mes 02, promedio s3: 1.6023612282355275	-	promedio s2: 6.703556724043851
a?o: 2007 - mes 03, promedio s3: 2.346871569703622	-	promedio s2: 6.5046484016219
a?o: 2007 - mes 04, promedio s3: 0.9731489221572055	-	promedio s2: 4.800339438153862
a?o: 2007 - mes 05, promedio s3: 1.6158602150537635	-	promedio s2: 5.1399641577060935
a?o: 2007 - mes 06, promedio s3: 1.6205714020900432	-	promedio s2: 4.375906573672869
a?o: 2007 - mes 07, promedio s3: 1.252173717675077	-	promedio s2: 3.478285290615381
a?o: 2007 - mes 08, promedio s3: 1.1141467569142498	-	promedio s2: 5.05271415123941
a?o: 2007 - mes 09, promedio s3: 1.7426038242511228	-	promedio s2: 5.24040464836335
a?o: 2007 - mes 10, promedio s3: 1.9694878802813747	-	prome

### *Identifique aquellos registros donde el consumo promedio del submetering 2 sea mayor al submetering 3.

In [54]:
#realizamos el join y revisamos
join_s2_s3 = s2_promedio.join(s3_promedio)
join_s2_s3.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[((2010, 8), (0.7212006200887369, 4.28152563211632))]

In [55]:
#ejecutamos nuestra funcion
registros_x_mayor_y(join_s2_s3, 2, 3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Registros donde el consumo promedio del submetering 2 es mayor al del submetering 3 (total: 0):